<a href="https://colab.research.google.com/github/Nour2806/solutions/blob/master/Benchmarks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Initialisation du benchmark

In [17]:
import numpy as np
import random
from numbers import Number
class SubspaceGenerator(object):

  def __init__ (self, n_samples, n_features_tot,n_label, n_cluster,cluster_per_subspace,subspaces_dim,weight_subspaces,
              weight_cluster,
              distributions, possible_distributions,nb_multi_distribution,
              outliers,compactness_factor,mv,mass,corr,rotate,alpha_n,random_state):
    ''' 
    Documentation
    Parameters:
    Out:
    '''
    self.random_state=random_state
    self.n_samples=n_samples
    self.n_features_tot=n_features_tot
    self.n_label=n_label
    self.n_cluster=n_cluster
    self.n_subspaces= len(weight_subspaces) if type(weight_subspaces) == list else weight_subspaces
    self.cluster_per_subspace= cluster_per_subspace
    self.n_cluster_per_subspace=cluster_per_subspace if type(cluster_per_subspace)!= list else [len(cluster_per_subspace[i]) if type(cluster_per_subspace[i])==list else  cluster_per_subspace[i] for i in range(len(cluster_per_subspace))]
    self.subspace_dim=subspaces_dim
    self.n_dim_per_subspace=subspaces_dim if type(subspaces_dim)!= list else [len(subspaces_dim[i]) if type(subspaces_dim[i])==list else  subspaces_dim[i] for i in range(len(subspaces_dim))]
    self.weight_subspaces= weight_subspaces 
    self.weight_cluster=weight_cluster
    self.distributions=distributions
    self.possible_distributions = possible_distributions if possible_distributions is not None \
              else ['gaussian', 'uniform']
    self.nb_multi_distribution=nb_multi_distribution
    self.compactness_factor = compactness_factor
    self.corr = corr
    self.rotate = rotate
    self.mv = mv
    self.alpha_n = alpha_n
    self.outliers = outliers


    # self._cmax = None
    # self.scale = scale
    # self.add_noise = add_noise
    # self.n_noise = n_noise if n_noise is not None else []
    # self.ki_coeff = ki_coeff
    self._validate_parameters_()
  



  def _validate_parameters_(self):
    # check and initialize parameters
    if self.n_samples is not None: ## check tous les parametres et leur initialisations
      if  type(self.n_samples)!= int or self.n_samples<1:
        raise ValueError("'n_samples' parameter must be int")
    else:
      self.n_samples=800
    print("samples",self.n_samples)

    if self.n_features_tot is not None:
      if  type(self.n_features_tot)!= int or self.n_features_tot<1:
        raise ValueError("'n_features_tot' parameter must be int")
    else:
      self.n_features_tot=3
    print("featurestot",self.n_features_tot)

    if self.n_label is not None:
      if  type(self.n_label)!= int or self.n_label<1:
        raise ValueError("'n_label' parameter must be int")
    else:
      self.n_label=1
    print("label",self.n_label)

    if self.n_cluster is not None:
      if self.n_cluster < 1 :
        raise ValueError('"n_classe" parameter must have at least one value!')
    else:
      self.n_cluster=1
    print('n_cluster',self.n_cluster)

    if self.n_subspaces is not None:
      if  type(self.n_subspaces)!= int or   self.n_subspaces<1:
        raise ValueError("'subspaces' parameter must be int > 1 or list of float  ")
    else:
      self.n_subspaces=1
    print('n_subspaces',self.n_subspaces)




    if self.n_cluster_per_subspace is not None:
        if type(self.n_cluster_per_subspace)==int:
          if self.n_cluster_per_subspace > self.n_cluster:
            raise ValueError("'cluster_per_subspace' must be less than n_cluster")
          else:
            self.n_cluster_per_subspace =[self.n_cluster_per_subspace for _ in range(self.n_subspaces)]  ## mm nb de cluster dans tous les sous-espaces ?
        else:
          if type(self.n_cluster_per_subspace)== list:
            if  len(self.n_cluster_per_subspace)!= self.n_subspaces or any(type(nb)!=int or nb>self.n_cluster for nb in self.n_cluster_per_subspace):
              raise ValueError(" length list of 'cluster_per_subspace' must be equal to subspaces and must be a list of list of int less than n_cluster")###
          else:
            raise ValueError("'cluster_per_subspace' input must be of type int or list of int")
    else:
        self.n_cluster_per_subspace=[random.randrange(1,self.n_cluster)  for i in range(self.n_subspaces)]
    print('n_cluster_per_subspace',self.n_cluster_per_subspace)



    if self.cluster_per_subspace is not None:
        if type(self.cluster_per_subspace)==int:
          if self.cluster_per_subspace > self.n_cluster:
            raise ValueError("'cluster_per_subspace' must be less than n_cluster")
          else:
            l=np.arange(self.n_cluster)
            np.random.shuffle(l)
            self.cluster_per_subspace =[[l[j] for j in range(self.n_cluster_per_subspace[i])] for i in range(self.n_subspaces)]
        else:
          if type(self.cluster_per_subspace)== list:
            if  len(self.cluster_per_subspace)!= self.n_subspaces:
              raise ValueError(" length list of 'cluster_per_subspace' must be equal to subspaces")###
            elif all(type(l)!=list for l in self.cluster_per_subspace):
              l=np.arange(self.n_cluster)
              np.random.shuffle(l)
              self.cluster_per_subspace =[[l[j] for j in range(self.n_cluster_per_subspace[i])] for i in range(self.n_subspaces)]
      
            else:
                if any(any(nb>self.n_cluster for nb in l) for l in self.cluster_per_subspace):
                  raise ValueError("cluster_per_subspace''must be of type list of list that all it's elements must be less than n_cluster ")
          else:
            raise ValueError("'cluster_per_subspace' input must be of type int or list ")
    else:
        l=np.arange(self.n_cluster)
        np.random.shuffle(l)
        self.cluster_per_subspace =[[ l[j] for j in range(self.n_cluster_per_subspace[i])] for i in range(self.n_subspaces)]
    print('cluster_per_subspace',self.cluster_per_subspace)

    


    if self.n_dim_per_subspace is not None:
        if type(self.n_dim_per_subspace)==int:
          if self.n_dim_per_subspace > self.n_features_tot:
            raise ValueError("'subspace_dim' must be less than n_features_tot")
          else:
            self.n_dim_per_subspace =[self.n_dim_per_subspace for _ in range(self.n_subspaces)]  ## mm nb de dimensions dans tous les sous-espaces ?
        else:
          if type(self.n_dim_per_subspace)== list:
            if  len(self.n_dim_per_subspace)!= self.n_subspaces or any(type(nb)!=int or nb>self.n_features_tot for nb in self.n_dim_per_subspace):
              raise ValueError(" length list of 'subspace_dim' must be equal to subspaces and must be list of list of int less than n_features_tot")###
          else:
            raise ValueError("'cluster_per_subspace' input must be of type int or list of int  ")
    else:
        self.n_dim_per_subspace=[random.randrange(1,self.n_features_tot)  for i in range(self.n_subspaces)]
    print('n_dim_per_subspace',self.n_dim_per_subspace)




    if self.subspace_dim is not None:
        if type(self.subspace_dim)==int:
          if self.subspace_dim > self.n_features_tot:
            raise ValueError("'subspace_dim' must be less than n_features_tot")
          else:
            l=np.arange(n_cluster)
            np.random.shuffle(l)
            self.subspace_dim =[[l[j] for j in range(self.n_dim_per_subspace[i])] for i in range(self.n_subspaces)]
        else:
          if type(self.subspace_dim)== list:
            if  len(self.subspace_dim)!= self.n_subspaces:
              raise ValueError(" length list of 'subspace_dim' must be equal to subspaces")###
            elif all(type(l)!=list for l in self.subspace_dim):
              l=np.arange(self.n_features_tot)
              np.random.shuffle(l)
              self.subspace_dim =[[l[j] for j in range(self.n_dim_per_subspace[i])] for i in range(self.n_subspaces)]
      
            else:
                if any(any(nb>self.n_features_tot for nb in l) for l in self.subspace_dim):
                  raise ValueError("subspace_dim''must be of type list of list that all it's elements must be less than n_features_tot ")
          else:
            raise ValueError("'subspace_dim' input must be of type int or list ")
    else:
        l=np.arange(self.n_features_tot)
        np.random.shuffle(l)
        self.subspace_dim =[[l[j] for j in range(self.n_dim_per_subspace[i])] for i in range(self.n_subspaces)]
    print('subspace_dim',self.subspace_dim)




    if self.weight_subspaces is not None:
        if type(self.weight_subspaces)== list:
          if  len(self.weight_subspaces)!= self.n_subspaces:
            raise ValueError(" dimension of 'weight_subspaces' must be equal to n_subspaces")
          else:
            if  not all(( 0.0 < nb < 1.0 ) for nb in self.weight_subspaces):
                raise ValueError("elements of 'weight_subspaces' must be in the intervall[0,1] and \
                sum of weight_subspaces must be equal to 1 ")
            if  sum(self.weight_subspaces)!=1:
                raise ValueError("sum of 'weight_subspaces' must be equal to 1.0")

        elif type(self.weight_subspaces)== int :
          self.weight_subspaces=[1/self.weight_subspaces for i in range(self.weight_subspaces )] ## weight equi probable ? 
        else:
    
          raise ValueError(" 'weight_subspaces' must be of type int or  list of float or None type")
    else:
      self.weight_subspaces=[1/self.n_subspaces for i in range(self.n_subspaces)] ## equi probable ? ou random ? ajout d'un parametre pour l'utilisateur?
    print('weight_subspaces',self.weight_subspaces)




    if self.weight_cluster is not None:
        if type(self.weight_cluster)==np.array:
          if self.weight_cluster.shape== (self.n_subspaces,self.n_cluster) :
            if  not all(all(( 0.0 < nb < 1.0 ) for nb in elem )for elem in self.weight_cluster ):
              raise ValueError("elements of 'weight_cluster' must be  in the intervall[0,1]in each subspace")
            if   any(sum(elem)!=1 for elem in self.weight_cluster):
              raise ValueError("sum of 'weight_cluster' must be equal to 1.0 in each subspace")
        else:
          raise ValueError(" 'weight_cluster' must have dimensions (n_subspaces,n_cluster)")
    else:
      self.weight_cluster=[[1/self.n_cluster_per_subspace[i] for j in range(self.n_cluster_per_subspace[i])] for i in range(self.n_subspaces)] ## à modifier
    print('weight_cluster',self.weight_cluster)

    # if max_min_class_per_subspace is not None:
    #   if type(max_min_class_per_subspace)!= list or  len(max_min_class_per_subspace)!=2:
    #     raise ValueError("'max_min_class_per_subspace' parameter must be a list with two elements [min,max]")
    # else:
    #   max_min_class_per_subspace=[1,n]
    
    
    
    


    if self.distributions is not None:
      # check validity of self.distributions, and turning it into a (n_clusters, n_feats) matrix
      if type(self.distributions)==list:
          if len(self.distributions)==self.n_subspaces:
            if all(type(self.distributions[i])== list for i in range(self.n_subspaces)):
              # if all(len(self.distributions[i])==self.n_cluster_per_subspace[i] for i in range(self.n_subspaces)):
              #   if not all(len(self.distributions[j][i])==self.n_dim_per_subspace[j][i] for i in range(len(self.n_dim_per_subspace[j])) for j in range(self.n_subspaces))):
              #     raise ValueError('distribution parameter must be of length n_dim per subspaces') 
              # else:
              #   if not all(len(self.distributions[j][i])==self.n_dim_per_subspace[j][i] for i in range(len(self.n_dim_per_subspace[j])) for j in range(self.n_subspaces))):
              #     raise ValueError('distributions parameter must be a list of list of length n_cluster_per_subspaces') # a reformuler
             ## à reformuler
    #   else:
    #     self.distributions = [self.distributions  for _ in range( self.n_subspaces)] #
    # else:
    #   self.distributions = [random.choice(self.possible_distributions) for _ in range( self.n_subspaces)] 

    # print(self.distributions)

    if self.compactness_factor is not None:
        if type(self.compactness_factor)==np.array:
          if self.compactness_factor.shape== (self.n_subspaces,self.n_cluster) :
            if  not all(all(( 0.0 < nb < 1.0 ) for nb in elem )for elem in self.compactness_factor ):
              raise ValueError("elements of 'compactness_factor' must be  in the intervall[0,1]in each subspace")
        else:
          if type(self.compactness_factor)== list:
            if len(self.compactness_factor)==self.n_subspaces:
              if  not all(( 0.0 < elem < 1.0 ) for elem in self.compactness_factor ):
                raise ValueError("elements of 'compactness_factor' must be  in the intervall[0,1]")
              else:
                self.compactness_factor=[[self.compactness_factor] * n_class for _ in range(self.n_subspaces) ]
            else:
              raise ValueError('"compactness factor" must be exactly of length of subspace!')
          if type(self.compactness_factor)== float:
            self.compactness_factor=[[self.compactness_factor for j in range(self.n_cluster)] for i in range(self.n_subspaces)] 
          else:
            raise ValueError(" 'compactness_factor' must have dimensions (n_subspaces,n_class) or list of float or float")
    else:
      self.compactness_factor=[[random.random() for j in range(self.n_cluster)] for i in range(self.n_subspaces)] ## à modifier
    
    print('compcatness',self.compactness_factor)



    if self.corr is not None:
        if type(self.corr)==np.array:
          if self.corr.shape== (self.n_subspaces,self.n_cluster) :
            if any(elem not in[True,False] for elem in self.corr):
              raise ValueError("elements of 'corr' must be  in the intervall[0,1]in each subspace")
        else:
          if type(self.corr)== list:
            if len(self.corr)==self.n_subspaces:
              self.corr=[[self.corr] * self.n_class for _ in range(self.n_subspaces) ]
            else:
              raise ValueError('"corr " must be exactly of length of subspace!')
          if type(self.corr)== float:
            self.corr=[[self.corr for j in range(self.n_cluster)] for i in range(self.n_subspaces)] 
          else:
            raise ValueError(" 'corr' must have dimensions (n_subspaces,n_class) or list of float or float")
    else:
      self.corr=[[random.choice([True, False]) for j in range(self.n_cluster)] for i in range(self.n_subspaces)]
    
    print('corr',self.corr)






    if self.rotate is not None:
        if type(self.rotate)==np.array:
          if self.rotate.shape== (self.n_subspaces,self.n_class) :
            if any(any(nb not in[True,False]  for nb in elem ) for elem in self.rotate):
              raise ValueError("elements of 'rotate' must be  False or True or None in each subspace")
        else:
          if type(self.rotate)== list:
            if len(self.rotate)==self.n_subspaces:
              if any(elem not in[True,False] for elem in self.rotate):
                raise ValueError("elements of 'rotate' must be  False or True or None")
              else:
                self.rotate=[[self.rotate] * self.n_cluster for _ in range(self.n_subspaces) ]
            else:
              raise ValueError('"rotate " must be exactly of length of subspace!')
          if type(self.rotate)== bool and self.rotate in[True,False]:
            self.rotate=[[self.rotate for j in range(self.n_cluster)] for i in range(self.n_subspaces)] 
          else:
            raise ValueError(" 'rotate' must be an array of dimension (n_subspaces,n_class) or \
            list of bool or bool in [True,False,None]")
    else:
      rotate=[[random.choice([True, False]) for j in range(self.n_cluster)] for i in range(self.n_subspaces)]
    print('rotate',self.rotate)


    if self.mv is not None:
        if self.mv.shape== (self.n_subspaces,self.n_class) :
          if any(any(nb not in[True,False]  for nb in elem ) for elem in self.mv):
            raise ValueError("elements of 'mv' must be a bool or None in each subspace")
        else:
          if type(self.mv)== list:
            if len(self.mv)==self.n_subspaces:
              if any(elem not in[True,False] for elem in self.mv):
                raise ValueError("elements of 'mv' must be bool or None")
              else:
                self.mv=[[self.mv] * self.n_cluster for _ in range(self.n_subspaces) ]
            else:
              raise ValueError('"mv " must be exactly of length of subspace!')
          if type(self.mv)== float and self.mv in[True,False]:
            self.mv=[[self.mv for j in range(self.n_cluster)] for i in range(self.n_subspaces)] 
          else:
            raise ValueError(" 'mv' must be an array of dimension (n_subspaces,n_class) or \
            list of bool or bool or None")
    else:
      mv=[[random.choice([True, False]) for j in range(self.n_cluster)] for i in range(self.n_subspaces)]
    print('mv',self.mv)



    if self.alpha_n is not None:
      # check validity of self.distributions, and turning it into a (n_clusters, n_feats) matrix
        if not (self.alpha_n.shape==(self.n_subspaces,self.n_features_tot) or len(self.alpha_n)==(self.n_subspaces) or \
                isinstance(self.alpha_n,Number) or self.alpha_n!=0):
          raise ValueError('Invalid alpha_n input! Input must have dimensions (n_subspaces,n_features).\
          or (,n_subspaces) or int and it must not be equal to 0')
        else: 
          if self.alpha_n.shape==(self.n_subspaces,self.n_cluster) :
            self.alpha_n = [self.alpha_n]*self.n_features_tot #
          if type(self.alpha_n)==str:
            self.alpha_n = [[self.alpha_n]*self.n_features_tot]*self.n_subspaces 
    else:
      self.alpha_n = [[random.randrange(1,10,1) for _ in range(self.n_features_tot)]  for _ in range(self.n_subspaces)] 
    print("alpha_n",self.alpha_n)
    
    if self.outliers is not None:
      # check validity of self.distributions, and turning it into a (n_clusters, n_feats) matrix
      if type(self.outliers)==list and  len(self.outliers)!=(self.n_subspaces):
        raise ValueError('Invalid outliers input! Input must have dimensions (,n_subspaces) or int ')
      else:
        if  not  isinstance(self.outliers,Number) :
          raise ValueError('Invalid outliers input! Input must have dimensions (,n_subspaces) or int ')
        else: 
          self.outliers = [self.outliers]*n_subspaces 
    else:
      self.outliers = [random.randrange(1,10,1)  for _ in range(self.n_subspaces)] 
    print('outliers',self.outliers)


  def _generate_subspaces(self):
    data :list =[]
    dimensions=[]
    # for i in range(n_subspaces):
    #   dimension=random.sample(list(np.arange(self.n_features)),subspace_dim[i])
    #   dimensions.append(dimension)
    #   gnr=ClusterGenerator(seed=1, n_samples=round(n_samples*weight_subspaces[i]), n_feats=subspace_dim[i],
    #                       n_feats_t= n_features,dimensions=dimension, k=n_class_per_subspace[i], min_samples=0, possible_distributions=None,
    #                         distributions='uniform', mv=None, corr=0.2, compactness_factor=1, alpha_n=1,
    #                         scale=True, outliers=5, rotate=True, add_noise=10, n_noise=None, ki_coeff=3.)
    #   X,y=gnr.generate_data()
    #   data.append((X,y))
    return data











### MGCDgen

#### Generate

In [ ]:
from __future__ import division
import math
import numpy as np
import scipy.linalg


def generate_mass(clus_cfg):
    """
    Get the number of samples to generate for each cluster.
    Args:
        clus_cfg (clusters.DataConfig): Configuration
    Returns:
        np.array: Array with len == nr of clusters, where each entry is the number of samples in the corresponding
            to generate in the corresponding cluster.
    """
    if type(clus_cfg.k) == list:
        mass = np.array(clus_cfg.k)
    else:
        mass = np.random.uniform(0, 1, clus_cfg.n_clusters)
        total_mass = mass.sum()
        mass = np.vectorize(math.floor)(clus_cfg.n_samples * mass / total_mass)
        abs_mass = mass.sum()
        
        if abs_mass < clus_cfg.n_samples:  # if samples are unassigned, send them to the cluster with least samples
            min_ind = np.argmin(mass)
            mass[min_ind] += clus_cfg.n_samples - abs_mass

        # guarantee there are enough samples in each cluster
        if clus_cfg.min_samples <= 0:
            min_mass = round(clus_cfg.n_samples / (clus_cfg.ki_coeff * clus_cfg.n_clusters))
        else:
            min_mass = clus_cfg.min_samples
        need_to_add = True
        while need_to_add:
            need_to_add = False
            min_ind = np.argmin(mass)
            if mass[min_ind] < min_mass:
                max_ind = np.argmax(mass)
                extra = min_mass - mass[min_ind]
                mass[max_ind] -= extra
                mass[min_ind] += extra
                need_to_add = True

    return mass.astype(dtype=float)


def locate_centroids(clus_cfg):
    """
    Generate locations for the centroids of the clusters.
    Args:
        clus_cfg (clusters.DataConfig): Configuration.
    Returns:
        np.array: Matrix (n_clusters, n_feats) with positions of centroids.
    """
    centroids = np.zeros((clus_cfg.n_clusters, clus_cfg.n_feats))

    p = 1.
    idx = 1
    for i, c in enumerate(clus_cfg._cmax):
        p *= c
        if p > 2 * clus_cfg.n_clusters + clus_cfg.outliers / clus_cfg.n_clusters:
            idx = i
            break
    idx += 1

    locis = np.arange(p)
    np.random.shuffle(locis)
    clin = locis[:clus_cfg.n_clusters]

    # voodoo magic for obtaining centroids
    res = clin
    for j in range(idx):
        center = ((res % clus_cfg._cmax[j]) + 1) / (clus_cfg._cmax[j] + 1)
        noise = (np.random.rand(clus_cfg.n_clusters) - 0.5) * clus_cfg.compactness_factor
        centroids[:, j] = center + noise
        res = np.floor(res / clus_cfg._cmax[j])
    for j in range(idx, clus_cfg.n_feats):
        center = np.floor(clus_cfg._cmax[j] * np.random.rand(clus_cfg.n_clusters) + 1) / (clus_cfg._cmax[j] + 1)
        noise = (np.random.rand(clus_cfg.n_clusters) - 0.5) * clus_cfg.compactness_factor
        centroids[:, j] = center + noise

    return centroids, locis, idx


def generate_clusters(clus_cfg, batch_size = 0):
    """
    Generate data.
    Args:
        clus_cfg (clusters.DataConfig): Configuration.
        batch_size (int): Number of samples for each batch.
    Yields:
        np.array: Generated samples.
        np.array: Labels for the samples.
    """
    # generate correlation and rotation matrices
    for cluster in clus_cfg.clusters:
        # generate random symmetric matrix with ones in the diagonal
        # uses the vine method described here
        # http://stats.stackexchange.com/questions/2746/how-to-efficiently-generate-random-positive-semidefinite-correlation-matrices
        # using the correlation input parameter to set a threshold on the values of the correlation matrix
        corr = np.eye(clus_cfg.n_feats)
        aux = np.zeros(corr.shape)

        beta_param = 4

        for k in range(clus_cfg.n_feats - 1):
            for i in range(k + 1, clus_cfg.n_feats):
                aux[k, i] = 2 * cluster.corr * (np.random.beta(beta_param, beta_param) - 0.5)
                p = aux[k, i]
                for l in range(k - 1, -1, -1):
                    p = p * np.sqrt((1 - aux[l, i]**2) * (1 - aux[l, k]**2)) + aux[l, i] * aux[l, k]
                corr[k, i] = p
                corr[i, k] = p
        perm = np.random.permutation(clus_cfg.n_feats)
        corr = corr[perm, :][:, perm]
        cluster.corr_matrix = np.linalg.cholesky(corr)
        cluster.correlation_matrix = corr

        # rotation matrix
        if cluster.rotate:
            cluster.rotation_matrix = get_rotation_matrix(clus_cfg.n_feats)

    if batch_size == 0:
        batch_size = clus_cfg.n_samples
    for batch in range(((clus_cfg.n_samples - 1) // batch_size) + 1):
        n_samples = min(batch_size, clus_cfg.n_samples - batch * batch_size)
        data, labels = compute_batch(clus_cfg, n_samples)
        yield data, np.reshape(labels, (len(labels), 1))


def get_rotation_matrix(n_feats):
    rot_mat = 2 * (np.random.rand(n_feats, n_feats) - 0.5)
    ort = scipy.linalg.orth(rot_mat)
    if ort.shape == rot_mat.shape:  # check if `rot_mat` is full rank, so that `ort` keeps the same shape
        return ort
    else:
        return get_rotation_matrix(n_feats)


def compute_batch(clus_cfg, n_samples):
    """
    Generates one batch of data.
    Args:
        clus_cfg (clusters.DataConfig): Configuration.
        n_samples (int): Number of samples in the batch.
    Returns:
        np.array: Generated sample.
    """
    # get probabilities of each class
    mass = clus_cfg.mass
    mass = np.insert(mass, 0, clus_cfg.outliers)  # class 0 is now the outliers (this changes to -1 further down)
    mass /= mass.sum()

    labels = np.random.choice(clus_cfg.n_clusters + 1, n_samples, p=mass) - 1
    # label -1 corresponds to outliers
    data = np.zeros((n_samples, clus_cfg.n_feats_t))
    # generate samples for each cluster
    for label in range(clus_cfg.n_clusters):
        cluster = clus_cfg.clusters[label]
        indexes = (labels == label)
        samples = sum(indexes)  # nr of samples in this cluster
        data[:,clus_cfg.dimensions][indexes,:] = cluster.generate_data(samples)
        data[:,clus_cfg.dimensions][indexes,:]= data[:,clus_cfg.dimensions][indexes,:].dot(cluster.corr_matrix)  # apply correlation to data

        # apply rotation
        if cluster.rotate:
            data[:,clus_cfg.dimensions][indexes,:] = data[:,clus_cfg.dimensions][indexes,:].dot(cluster.rotation_matrix)

        # add centroid
        data[:,clus_cfg.dimensions][indexes,:] += clus_cfg._centroids[label]


        
        # add noisy variables
        for d in [i for i in range(clus_cfg.n_feats_t ) if i not in clus_cfg.dimensions] :
            data[:, d] = np.random.rand(n_samples)

    # generate outliers
    indexes = (labels == -1)
    out = sum(indexes)

    # voodoo magic for generating outliers
    locis = clus_cfg._locis[clus_cfg.n_clusters:]
    res = locis[np.arange(out) % len(locis)]
    for j in range(clus_cfg._idx):
        center = ((res % clus_cfg._cmax[j]) + 1) / (clus_cfg._cmax[j] + 1)
        noise = (1 / (clus_cfg._cmax[j] + 1)) * np.random.rand(out) - (1 / (2 * (clus_cfg._cmax[j] + 1)))
        data[indexes, j] = center + noise
        res = np.floor(res / clus_cfg._cmax[j])
    for j in range(clus_cfg._idx, clus_cfg.n_feats):
        center = np.floor(clus_cfg._cmax[j] * (np.random.rand(out) + 1)) / (clus_cfg._cmax[j] + 1)
        noise = (1 / (clus_cfg._cmax[j] + 1)) * np.random.rand(out) - (1 / (2 * (clus_cfg._cmax[j] + 1)))
        data[indexes, j] = center + noise

    return data, labels

#### Init

In [1]:
from numbers import Number
import random
import math
import six
import numpy as np


class ClusterGenerator(object):
    """
    Structure to handle the input and create clusters according to it.
    """
    def  __init__(self, seed=1, n_samples=2000, n_feats=2, k=5, min_samples=0, possible_distributions=None,
                  distributions=None, nb_multi_distribution=[1,2,3],mv=True, corr=0., compactness_factor=0.1, alpha_n=1,
                  scale=True, outliers=50, rotate=True, add_noise=0, n_noise=None, ki_coeff=3.,type_rel_clusters='imbriqué', **kwargs):
        """
        Args:
            seed (int): Seed for the generation of random values. Useful for consistency.
            n_samples (int): Number of samples to generate.
            n_feats (int): Number of dimensions/features for each sample.
            k (int or list of int): Number of clusters to generate. If input is a list, each element in it specifies the
                number of samples in each cluster. In that case, the number of clusters will be the length of the list.
            min_samples (int): Minimum number of samples in each cluster. If 0, the default minimum for a cluster with
                :math:`N` samples is :math:`N/(\\text{ki_coeff}*k)`.
            possible_distributions (list): List of distributions to randomly choose from. Each element in this list
                must either be a valid str (valid str are defined in :data:`~.distributions.valid_distributions`
                OR a function which implements the distribution OR an instance of
                :class:`~.distributions.Distribution`.
                This parameter is overridden by ``distributions``, when set.
            distributions (str or function or .distributions.Distribution or list): Distribution to be used.
                If list, its length must be ``k``, and each element in the list must either be a valid str (indicating
                the distribution to be used) OR a function which implements the distribution OR a list of str/functions
                with length ``n_feats``.
                Instances of :class:`~.distributions.Distribution` can also be used.
                Valid str are defined in :data:`~.distributions.valid_distributions`.
            mv (bool or list of bool or None): Multivariate distributions or distributions defining intra-distances. If
                True, distributions define feature values (multivariate). If False, distributions define
                intra-distances.
                If None, this choice is made at random.
                If a list, its length must be ``k``, and each value in the list applies to one cluster.
            corr (float or list of float): Maximum (in absolute value) correlation between variables.
                If a list, its length must be ``k``, and each value in the list applies to one cluster.
            compactness_factor (float or list of float): Compactness factor.
                If a list, its length must be ``k``, and each value in the list applies to one cluster.
            alpha_n (float or list of float): Determines grid hyperplanes. If :math:`\\alpha_n > 0`, the number of
                hyperplanes is a factor of
                :math:`\\alpha_n * \\left \\lfloor{1 + \\frac{k}{\\log(k)}}\\right \\rfloor`.
                If :math:`\\alpha_n < 0`, the number of hyperplanes is :math:`|\\alpha_n|`.
                If a list, its length must be ``n_feats``, and each value in the list applies to one dimension.
            scale (bool or list of bool): Optimizes cluster separation based on grid size. If True, scale based on min
                distance between grid hyperplanes. If False, scale based on max distance between grid hyperplanes.
                If None, does not scale.
                If a list, its length must be ``k``, and each value in the list applies to one cluster.
            outliers (int): Number of outliers.
            rotate (bool or list of bool): If True, clusters can rotate.
                If a list, its length must be ``k``, and each value in the list applies to one cluster.
            add_noise (int): Add this number of noisy dimensions.
            n_noise (list): Parameter that manages noisy dimensions.
                If a list of int (of size :math:`\\leq` ``n_feats``, and each element is :math:`\\geq 0` and
                :math:`<` ``n_feats``), each dimension listed (0-indexed) will have only noise.
                If a list of list of int (of length ``k``, and each element is a list of length :math:`\\leq`
                ``n_feats``, with values :math:`\\geq 0` and :math:`<` ``n_feats``), each list indicates the noisy
                dimensions for a particular cluster.
            ki_coeff (float): Coefficient used to define the default minimum number of samples per cluster.
        """
        self.seed = seed
        self.n_samples = n_samples
        self.n_feats = n_feats
        self.k = k
        self.n_clusters = len(k) if type(k) == list else k
        self.min_samples = min_samples
        self.possible_distributions = possible_distributions if possible_distributions is not None \
            else ['gaussian', 'uniform']
        self.distributions = distributions
        self.nb_multi_distribution=nb_multi_distribution
        self.mv = mv
        self.corr = corr
        self.compactness_factor = compactness_factor
        self.alpha_n = alpha_n
        self._cmax = None
        self.scale = scale
        self.outliers = outliers
        self.rotate = rotate
        self.add_noise = add_noise
        self.n_noise = n_noise if n_noise is not None else []
        self.ki_coeff = ki_coeff
        self.type_rel_clusters=type_rel_clusters
  

        random.seed(self.seed)

        for key, val in kwargs.items():
            self.__dict__[key] = val

        self._distributions = None
        self._validate_parameters()
        self.clusters = self.get_cluster_configs()

        self._mass = None
        self._centroids = None
        self._locis = None
        self._idx = None

    def generate_data(self, batch_size=0):
        np.random.seed(self.seed)
        self._mass = generate_mass(self)
        self._centroids, self._locis, self._idx = locate_centroids(self)
        batches = generate_clusters(self, batch_size)
        if batch_size == 0:  # if batch_size == 0, just return the data instead of the generator
            return next(batches)
        else:
            return batches

    def get_cluster_configs(self):
        return [Cluster(self, i) for i in range(self.n_clusters)]

    def _validate_parameters(self):
        """
        Method to validate the parameters of the object.
        """
        if hasattr(self.k, '__iter__'):
            if len(self.k) == 1:  # if only one input, no point in being a list
                self.k = self.k[0]
                self.n_clusters = self.k
            elif len(self.k) < 1:
                raise ValueError('"k" parameter must have at least one value!')
            else:
                if sum(self.k) != self.n_samples:
                    raise ValueError('Total number of points must be the same as the sum of points in each cluster!')

        if self.distributions is not None:
            # check validity of self.distributions, and turning it into a (n_clusters, n_feats) matrix
            if hasattr(self.distributions, '__iter__') and not type(self.distributions) == str:
                if len(self.distributions) != self.n_clusters:
                    raise ValueError('There must be exactly one distribution input for each cluster!')
                if hasattr(self.distributions[0], '__iter__'):
                    if not all(hasattr(elem, '__iter__') and len(elem) == self.n_feats for elem in self.distributions):
                        raise ValueError('Invalid distributions input! Input must have dimensions (n_clusters, n_feats).')
            else:
                self.distributions = [self.distributions] * self.n_clusters
            self._distributions = check_input(self.distributions)
        else:
            self.distributions = [random.choice(self.possible_distributions) for _ in range(self.n_clusters)]
            self._distributions = check_input(self.distributions)

        # check validity of self.mv, and turn it into a list with self.n_clusters elements
        if hasattr(self.mv, '__iter__'):
            if len(self.mv) != self.n_clusters:
                raise ValueError('There must be exactly one "mv" parameter for each cluster!')
        else:
            if self.mv is None:
                self.mv = [random.choice([True, False]) for _ in range(self.n_clusters)]
            else:
                self.mv = [self.mv] * self.n_clusters
        assert all(_validate_mv(elem) for elem in self.mv)


        # check validity of self.scale, and turn it into a list with self.n_clusters elements
        if hasattr(self.scale, '__iter__'):
            if len(self.scale) != self.n_clusters:
                raise ValueError('There must be exactly one "scale" parameter for each cluster!')
        else:
            self.scale = [self.scale] * self.n_clusters
        assert all(_validate_scale(elem) for elem in self.scale)

        # check validity of self.corr, and turn it into a list with self.n_clusters elements
        if hasattr(self.corr, '__iter__'):
            if len(self.corr) != self.n_clusters:
                raise ValueError('There must be exactly one correlation "corr" value for each cluster!')
        else:
            self.corr = [self.corr] * self.n_clusters
        assert all(_validate_corr(elem) for elem in self.corr)

        # check validity of self.alpha_n, and turn it into a list with self.n_feats elements
        if hasattr(self.alpha_n, '__iter__'):
            if len(self.alpha_n) != self.n_feats:
                raise ValueError('There must be exactly one hyperplane parameter "alpha_n" value for each dimension!')
        else:
            self.alpha_n = [self.alpha_n] * self.n_feats
        assert all(_validate_alpha_n(elem) for elem in self.alpha_n)

        # set self._cmax
        self._cmax = [math.floor(1 + self.n_clusters / math.log(self.n_clusters))] * self.n_feats \
            if self.n_clusters > 1 else [1 + 2 * (self.outliers > 1)] * self.n_feats
        self._cmax = [round(-a) if a < 0 else round(c * a) for a, c in zip(self.alpha_n, self._cmax)]
        self._cmax = np.array(self._cmax)

        # check validity of self.compactness_factor, and turn it into a list with self.n_clusters elements
        if hasattr(self.compactness_factor, '__iter__'):
            if len(self.compactness_factor) != self.n_clusters:
                raise ValueError('There must be exactly one compactness "compactness_factor" value for each cluster!')
        else:
            self.compactness_factor = [self.compactness_factor] * self.n_clusters
        assert all(_validate_compactness_factor(elem) for elem in self.compactness_factor)

        cmax_max = max(self._cmax)
        cmax_min = min(self._cmax)
        self.compactness_factor = [cp / cmax_max if s else (cp / cmax_min if not s else cp)
                                   for cp, s in zip(self.compactness_factor, self.scale)]

        # check validity of self.rotate, and turn it into a list with self.n_clusters elements
        if hasattr(self.rotate, '__iter__'):
            if len(self.rotate) != self.n_clusters:
                raise ValueError('There must be exactly one rotate value for each cluster!')
        else:
            self.rotate = [self.rotate] * self.n_clusters
        assert all(_validate_rotate(elem) for elem in self.rotate)

        # check validity of self.add_noise and self.n_noise
        if not isinstance(self.add_noise, six.integer_types):
            raise ValueError('Invalid input for "add_noise"! Input must be integer.')
        if hasattr(self.n_noise, '__iter__'):
            if len(self.n_noise) == 0:
                self.n_noise = [[]] * self.n_clusters
            if hasattr(self.n_noise[0], '__iter__'):
                if len(self.n_noise) != self.n_clusters:
                    raise ValueError('Invalid input for "n_noise"! List length must be the number of clusters.')
            else:
                self.n_noise = [self.n_noise] * self.n_clusters
        else:
            raise ValueError('Invalid input for "n_noise"! Input must be a list.')
        assert all(_validate_n_noise(elem, self.n_feats) for elem in self.n_noise)

    @property
    def mass(self):
        return self._mass


class Cluster(object):
    """
    Contains the parameters of an individual cluster.
    """

    settables = ['distributions', 'mv', 'corr', 'compactness_factor', 'scale', 'rotate', 'n_noise']
    """
    List of settable properties of Cluster. These are the parameters which can be set at a cluster level, and override
    the parameters of the cluster generator.
    """

    def __init__(self, cfg, idx, corr_matrix=None):
        """
        Args:
            cfg (ClusterGenerator): Configuration of the data.
            idx (int): Index of a cluster.
            corr_matrix (np.array): Valid correlation matrix to use in this cluster.
        """
        self.cfg = cfg
        self.idx = idx
        self.corr_matrix = corr_matrix

    def generate_data(self, samples):          
        if hasattr(self.distributions, '__iter__'):
            out = np.zeros((samples, self.cfg.n_feats))
            for f in range(self.cfg.n_feats):
              o=np.zeros(samples,1)
              m=[]
              for nb in range(self.nb_multi_distribution):
                o+=self.distributions[f]((samples,1), self.mv, self.compactness_factor) 
                
                [[m.append(i) for i in j]for j in o]
                out[:,f] = m
            return out
        else:
            out=np.zeros((samples, self.cfg.n_feats))
            for nb in range(self.nb_multi_distribution):
              out +=self.distributions((samples, self.cfg.n_feats), self.mv, self.compactness_factor)
            return out

    @property
    def n_feats(self):
        return self.cfg.n_feats

    @property
    def distributions(self):
        return self.cfg._distributions[self.idx]
    
    @property
    def nb_multi_distribution(self):
        return self.cfg.nb_multi_distribution[self.idx]

    @distributions.setter
    def distributions(self, value):
        if isinstance(value, six.string_types):
            self.cfg._distributions[self.idx] = dist.get_dist_function(value)
        elif hasattr(value, '__iter__'):
            self.cfg._distributions[self.idx] = [dist.get_dist_function(d) for d in value]
        else:
            self.cfg._distributions[self.idx] = dist.get_dist_function(value)

    @property
    def mv(self):
        return self.cfg.mv[self.idx]

    @mv.setter
    def mv(self, value):
        assert _validate_mv(value)
        self.cfg.mv[self.idx] = value

    @property
    def corr(self):
        return self.cfg.corr[self.idx]

    @corr.setter
    def corr(self, value):
        assert _validate_corr(value)
        self.cfg.corr[self.idx] = value

    @property
    def compactness_factor(self):
        return self.cfg.compactness_factor[self.idx]

    @compactness_factor.setter
    def compactness_factor(self, value):
        assert _validate_compactness_factor(value)
        self.cfg.compactness_factor[self.idx] = value

    @property
    def scale(self):
        return self.cfg.scale[self.idx]

    @scale.setter
    def scale(self, value):
        assert _validate_scale(value)
        self.cfg.scale[self.idx] = value

    @property
    def rotate(self):
        return self.cfg.rotate[self.idx]

    @rotate.setter
    def rotate(self, value):
        assert _validate_rotate(value)
        self.cfg.rotate[self.idx] = value

    @property
    def n_noise(self):
        return self.cfg.n_noise[self.idx]

    @n_noise.setter
    def n_noise(self, value):
        assert _validate_n_noise(value, self.cfg.n_feats)
        self.cfg.n_noise[self.idx] = value


class ScheduledClusterGenerator(ClusterGenerator):
    """
    This cluster generator takes a schedule and all the ClusterGenerator arguments, and activates only the specified
    clusters in the schedule, for each time step.
    A time step is defined as one get call to ``self.mass``, which is done when generating each new batch.
    That is, one time step is one call to :func:`.generate.compute_batch`.
    """
    def __init__(self, schedule, *args, **kwargs):
        """
        Args:
            schedule (list): List in which each element contains the indexes of the clusters active in the respective
                time step.
            *args: args for :meth:`ClusterGenerator.__init__`.
            **kwargs: kwargs for :meth:`ClusterGenerator.__init__`.
        """
        super(ScheduledClusterGenerator, self).__init__(*args, **kwargs)
        self.cur_time = 0
        self.schedule = schedule

    @property
    def mass(self):
        mass = self._mass.copy()
        cur_clusters = self.schedule[self.cur_time % len(self.schedule)]
        for c in range(len(mass)):  # set the mass of clusters not scheduled now to 0
            if c not in cur_clusters:
                mass[c] = 0

        self.cur_time += 1  # increase time
        return mass


def _validate_mv(mv):
    """
    Checks validity of input for `mv`.
    Args:
        mv (bool): Input to check validity
    Returns:
        bool: True if valid. Raises exception if not.
    """
    if mv not in [True, None, False]:
        raise ValueError('Invalid input value for "mv"!')
    return True


def _validate_corr(corr):
    """
    Checks validity of input for `corr`.
    Args:
        corr (float): Input to check validity.
    Returns:
        bool: True if valid. Raises exception if not.
    """
    if not isinstance(corr, Number):
        raise ValueError('Invalid input value for "corr"! Values must be numeric')
    if not 0 <= corr <= 1:
        raise ValueError('Invalid input value for "corr"! Values must be between 0 and 1.')
    return True


def _validate_compactness_factor(compactness_factor):
    """
    Checks validity of input for `compactness_factor`.
    Args:
        compactness_factor (float): Input to check validity.
    Returns:
        bool: True if valid. Raises exception if not.
    """
    if not isinstance(compactness_factor, Number):
        raise ValueError('Invalid input value for "compactness_factor"! Values must be numeric')
    # TODO 0 <= compactness_factor <= 1 ?
    return True


def _validate_alpha_n(alpha_n):
    """
    Checks validity of input for `alpha_n`.
    Args:
        alpha_n (float): Input to check validity.
    Returns:
        bool: True if valid. Raises exception if not.
    """
    if not isinstance(alpha_n, Number):
        raise ValueError('Invalid input for "alpha_n"! Values must be numeric.')
    if alpha_n == 0:
        raise ValueError('Invalid input for "alpha_n"! Values must be different from 0.')
    return True


def _validate_scale(scale):
    """
    Checks validity of input for `scale`.
    Args:
        scale (bool): Input to check validity.
    Returns:
        bool: True if valid. Raises exception if not.
    """
    if scale not in [True, None, False]:
        raise ValueError('Invalid input value for "scale"! Input must be boolean (or None).')
    return True


def _validate_rotate(rotate):
    """
    Checks validity of input for `rotate`.
    Args:
        rotate (bool): Input to check validity.
    Returns:
        bool: True if valid. Raises exception if not.
    """
    if rotate not in [True, False]:
        raise ValueError('Invalid input for "rotate"! Input must be boolean.')
    return True


def _validate_n_noise(n_noise, n_feats):
    """
    Checks validity of input for `n_noise`.
    Args:
        n_noise (list of int): Input to check validity.
        n_feats (int): Number of dimensions/features.
    Returns:
    """
    if not hasattr(n_noise, '__iter__'):
        raise ValueError('Invalid input for "n_noise"! Input must be a list.')
    if len(n_noise) > n_feats:
        raise ValueError('Invalid input for "n_noise"! Input has more dimensions than total number of dimensions.')
    if not all(isinstance(n, six.integer_types) for n in n_noise):
        raise ValueError('Invalid input for "n_noise"! Input dimensions must be integers.')
    if not all(0 <= n < n_feats for n in n_noise):
        raise ValueError('Invalid input for "n_noise"! Input dimensions must be in the interval [0, "n_feats"[.')
    return True

#### Distributions

In [ ]:
from __future__ import division
from six import integer_types, string_types
import math
import numpy as np


distributions_list = {
    'uniform': lambda shape, param: np.random.uniform(-param, param, shape),
    'gaussian': lambda shape, param: np.random.normal(0, param, shape),
    'logistic': lambda shape, param: np.random.logistic(0, param, shape),
    'triangular': lambda shape, param: np.random.triangular(-param, 0, param, shape),
    'gamma': lambda shape, param: np.random.gamma(2 + 8 * np.random.rand(), param / 5, shape),
    'gap': lambda shape, param: gap(shape, param),
    'binaire': lambda shape,param : np.random.choice([0,1],shape)
}
"""List of distributions for which you can just provide a string as input."""

# Aliases for distributions should be put here.
distributions_list['normal'] = distributions_list['gaussian']

valid_distributions = distributions_list.keys()
"""List of valid strings for distributions."""


def _validate_shape(shape):

    if not (hasattr(shape, '__iter__') and (len(shape) == 2 or len(shape) == 1))\
            and not isinstance(shape, integer_types):
        raise ValueError('Error! "shape" must be an integer or a tuple with size 2!')
    return True


def _validate_shape_intradistance(shape):
  
    if not (hasattr(shape, '__iter__') and len(shape) == 2):
        raise ValueError('Error! "shape" must be a tuple with size 2!')
    return True


def _aux_rms(mat):
    return np.sqrt((mat**2.).sum(1) / mat.shape[1]).reshape((mat.shape[0], 1))


def _intradistance_aux(shape):
    assert _validate_shape_intradistance(shape)
    out = np.random.rand(*shape) - 0.5
    out = math.sqrt(shape[1]) * out / _aux_rms(out)
    return out


class Distribution(object):
    def __init__(self, f, **kwargs):
        self.f = f
        self.kwargs = kwargs

    def __call__(self, shape, intra_distance, *args, **kwargs):
        new_kwargs = self.kwargs.copy()
        new_kwargs.update(kwargs)  # add keyword arguments given in __init__
        if intra_distance:
            assert _validate_shape_intradistance(shape)
            out = _intradistance_aux(shape)
            return out * self.f((shape[0], 1), *args, **new_kwargs)
        else:
            assert _validate_shape(shape)
            return self.f(shape, *args, **new_kwargs)


def gap(shape, param):
    out = np.zeros(shape)
    for j in range(shape[1]):
        new_shape = (2 * shape[0], 1)
        aux = np.random.normal(0, param, new_shape)
        med_aux = _aux_rms(aux)
        median = np.median(med_aux)
        out[:, j] = aux[med_aux > median][:shape[0]]
    return out


def check_input(distributions):
    """
    Checks if the input distributions are valid. That is, check if they are either strings or functions. If they are
    strings, also check if they are contained in `distributions_list`.
    Args:
        distributions (list of list of (str or function)): Distributions given as input.
    Returns:
        (list of list of function): Functions for the distributions given as input.
    """
    return [[get_dist_function(d) for d in l] if hasattr(l, '__iter__') and not isinstance(l, string_types) else get_dist_function(l)
            for l in distributions]


def get_dist_function(d):
    """
    Transforms distribution name into respective function.
    Args:
        d (str or function): Input distribution str/function.
    Returns:
        function: Actual function to compute the intended distribution.
    """
    if isinstance(d, Distribution):
        return d
    elif hasattr(d, '__call__'):
        return Distribution(d)
    elif isinstance(d, string_types):
        try:
            return Distribution(distributions_list[d])
        except KeyError:
            raise ValueError('Invalid distribution name "' + d + '". Available names are: ' +
                             ', '.join(distributions_list.keys()))
    else:
        raise ValueError('Invalid distribution input!')
    # TODO also check for None (and provide random choice)

# lancer



In [ ]:
n_subspaces=3
n_cluster=10
n_features_tot=3
gnr=SubspaceGenerator(n_samples=1800, n_features_tot=n_features_tot, n_label=None, n_cluster=n_cluster,cluster_per_subspace=[1,10,3],
                      subspaces_dim=[3,2,3], weight_subspaces=3,weight_cluster=None,n_informative=None,
                      n_redundant=None, n_repeated=None,distributions='gaussian',nb_multi_distribution=None, possible_distributions=None,
                      outliers=10,compactness_factor=None,mv=None,mass=None,corr=None,rotate=False,alpha_n=None,random_state=1)
# X,y=list(zip(*gnr))

samples 1800
featurestot 3
label 1
n_cluster 10
n_subspaces 3
n_cluster_per_subspace [1, 10, 3]
cluster_per_subspace [[9], [9, 6, 7, 5, 1, 8, 4, 0, 2, 3], [9, 6, 7]]
n_dim_per_subspace [3, 2, 3]
subspace_dim [[1, 0, 2], [1, 0], [1, 0, 2]]
weight_subspaces [0.3333333333333333, 0.3333333333333333, 0.3333333333333333]
weight_cluster [[1.0], [0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1], [0.3333333333333333, 0.3333333333333333, 0.3333333333333333]]
n_info [[0.86], [0.89, 0.83, 0.85, 0.86, 0.84, 0.91, 0.94, 0.88, 0.9, 0.89], [0.96, 0.85, 0.97]]
n_redund [[0.06], [0.05, 0.01, 0.05, 0.05, 0.11, 0.0, 0.0, 0.09, 0.05, 0.02], [0.0, 0.07, 0.02]]
n_repeated [[0.0], [0.01, 0.11, 0.07, 0.07, 0.04, 0.07, 0.04, 0.02, 0.02, 0.04], [0.04, 0.07, 0.0]]
compcatness [[0.46409885942736995, 0.8593439945925122, 0.037540205045941466, 0.8806646761641025, 0.09978083712900598, 0.47615130643079495, 0.3806404829427804, 0.9153203574966597, 0.8946769050477755, 0.9750150462962449], [0.8095783404579836, 0.772777736

### Mes tests

In [5]:
import pandas as pd
# Visualisation 
from matplotlib import pyplot as plt
from matplotlib import patches as ptc
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
import plotly.graph_objects as go

opacity: list = [1 for i in range(n_class)]


# sc: StandardScaler = StandardScaler() # parametre à demander à l'utilisateur
# X: np.ndarray = sc.fit_transform(X)
# pca: PCA = PCA(n_components=X.shape[1]) # nombre de dimensions -- > à demander à l'utilisateur
# X: np.ndarray = pca.fit_transform(X)
for (k,l,m) in zip(np.arange(n_class),np.arange(n_class),np.arange(n_class)):
  if k!=(l or m) and l !=(k or m) and m!=(k or l): 
    data: list = []
    for j in range(n_subspaces):
      name: list = {i:str('%d.%d' % (j, i)) for i, val in zip(range(n_class), np.arange(n_class))} # num des clusters
      data_plot: pd.DataFrame = pd.DataFrame(X[j])
      data_plot["prediction"] = y[j]
          # data_plot["prediction"] = prediction.replace([-1,1], name)

      for i, val in zip(range(n_class), np.arange(n_class)):
            data_semi_plot: pd.DataFrame = data_plot[data_plot["prediction"] == val]
          #  data.append(go.Scatter3d(x=data_semi_plot[0], y=data_semi_plot[2], z=data_semi_plot[3], name=name[val], mode='markers',marker=dict(size=6), opacity=opacity[i]))
            data.append(go.Scatter3d(x=data_semi_plot[k], y=data_semi_plot[l], z=data_semi_plot[m], name=name[val], mode='markers',marker=dict(size=6), opacity=opacity[i],title="%d %d %d" %(k,l,m)))
    fig: go.Figure = go.Figure(data)
    fig.show()


NameError: ignored

In [ ]:
import pandas as pd
# Visualisation 
from matplotlib import pyplot as plt
from matplotlib import patches as ptc
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
import plotly.graph_objects as go

opacity: list = [1 for i in range(n_class)]


# sc: StandardScaler = StandardScaler() # parametre à demander à l'utilisateur
# X: np.ndarray = sc.fit_transform(X)
# pca: PCA = PCA(n_components=X.shape[1]) # nombre de dimensions -- > à demander à l'utilisateur
# X: np.ndarray = pca.fit_transform(X)
data: list = []
for j in range(n_subspaces):
  name: list = {i:str('%d.%d' % (j, i)) for i, val in zip(range(n_class), np.arange(n_class))} # num des clusters
  data_plot: pd.DataFrame = pd.DataFrame(X[j])
  data_plot["prediction"] = y[j]
      # data_plot["prediction"] = prediction.replace([-1,1], name)

  for i, val in zip(range(n_class), np.arange(n_class)):
        data_semi_plot: pd.DataFrame = data_plot[data_plot["prediction"] == val]
        #data.append(go.Scatter3d(x=data_semi_plot[0], y=data_semi_plot[2], z=data_semi_plot[3], name=name[val], mode='markers',marker=dict(size=6), opacity=opacity[i]))
        data.append(go.Scatter3d(x=data_semi_plot[0], y=data_semi_plot[1], z=data_semi_plot[4], name=name[val], mode='markers',marker=dict(size=6), opacity=opacity[i]))

fig: go.Figure = go.Figure(data)
fig.show()

distributions on the real line:
------------------------------
       uniform
       triangular
       normal (Gaussian)
       lognormal
       negative exponential
       gamma
       beta
       pareto
       Weibull

distributions on the circle (angles 0 to 2pi)
---------------------------------------------
       circular uniform
       von Mises